In [ ]:
from nimlab import datasets as nimds
from matplotlib import pyplot as plt
import numpy as np
from nilearn import image, plotting, maskers
import nibabel as nib
import os
import pandas as pd
from tqdm import tqdm


In [ ]:
#User Input Below
path_to_paths_csv = r"C:\Users\calvin.howard\Dropbox (Partners HealthCare)\memory\functional_networks\file_paths\paths_to_adni_t_maps.csv"
#path_to_paths_csv = r"C:\Users\calvin.howard\Dropbox (Partners HealthCare)\memory\functional_networks\file_paths\<your_file_name_here>.csv"

out_dir = r"C:\Users\calvin.howard\Dropbox (Partners HealthCare)\memory\functional_networks\seeds"

In [ ]:
#preparing
if not os.path.exists(out_dir):
    os.makedirs(out_dir)

paths_df = pd.read_csv(path_to_paths_csv)
print("I found", len(paths_df), "participants:")
paths_df.head()

In [ ]:
mask = nimds.get_img("mni_icbm152")
threshold = 0.95

In [ ]:
def binarize_function(matrix, tails=2, threshold=0.95, abs_val=False):
    seed_matrix = np.zeros(matrix.shape)
    if tails == 2:
        seed_matrix[matrix>threshold] = 1 #one out from [threshold inf]
        seed_matrix[matrix<-threshold] = -1 #one out from matrix <-threshold
        seed = 'two_tail_'
    else:
        seed_matrix[matrix>threshold] = 1
    if abs_val == True:
        print('develop this')
        seed = seed + 'absval_'
    return seed_matrix, seed

seed_matrices = {}
seed = ''
for index, data in paths_df.iterrows():
    path = data.values[0]
    img = image.load_img(path)
    img_matrix = img.get_fdata()
    img_matrix = np.nan_to_num(img_matrix, nan=0, posinf=0, neginf=0)

    seed_matrix, seed = binarize_function(img_matrix, tails=2, threshold=0.95)
    seed_matrices[f'{path}'] = seed_matrix

print('Test Image')
t = image.new_img_like(mask, seed_matrices[f'{paths_df.iat[1,0]}'])
plotting.view_img(t, cut_coords=(0,0,0), opacity=0.75)

In [ ]:
files = {}
for index, data in paths_df.iterrows():
    path = data.values[0]
    matrix = seed_matrices[path]
    matrix_img = image.new_img_like(mask, matrix)
    files[path] = matrix_img
    plotting.plot_stat_map(matrix_img, display_mode="z", cut_coords=[-42,-24,-12,0,12,24,42,54], cmap="cold_hot", colorbar=True,title=os.path.basename(path), vmax=6)


In [ ]:
#Finish up and save
print(files)
for k, v in files.items():
    seedname = seed + os.path.basename(k)
    v.to_filename(os.path.join(out_dir, seedname))
    pass